In [105]:
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn import metrics, utils

In [110]:
path = Path("../mini_metrics/mini_results.csv")

In [111]:
df = pd.read_csv(path)
df

,instance_id,filename,level,label,prediction,confidence,threshold,prediction_made,correct
0,0,4ef6196e3bc1.jpg,0,1799135,5112283,0.258231,0.5,1,0
1,0,4ef6196e3bc1.jpg,1,1798902,8407958,0.086433,0.5,1,0
2,0,4ef6196e3bc1.jpg,2,7015,7054,0.998957,0.5,1,0
3,1,abd3c76abce0.jpg,0,1799135,1773163,0.042152,0.5,1,1
4,1,abd3c76abce0.jpg,1,1798902,1772926,0.169087,0.5,1,1
5,1,abd3c76abce0.jpg,2,7015,7015,0.999116,0.5,1,1
6,2,ieb2h8nwo94j.jpg,0,1799135,1773163,0.042152,0.5,0,1
7,2,ieb2h8nwo94j.jpg,1,1798902,1772926,0.169087,0.5,0,1
8,2,ieb2h8nwo94j.jpg,2,7015,7015,0.234959,0.5,0,1
9,3,j9ebuiqexu7b.jpg,0,1799135,1773163,0.042152,0.5,1,1


In [112]:
# if the prediction is not made put 0, if the prediction is made, either put 1 or -1
# rewards = lambda level: (3-level)/6
rewards = pd.Series([1/2 - x/6 for x in range(3)])
# penalties = lambda level: (-1-level)/6)
penalties = pd.Series([-(1+x)/6 for x in range(3)])

m = np.where(
    df['confidence']>df['threshold'], 
    np.where(
        df['label']==df['prediction'],
        rewards[df['level']], 
        penalties[df['level']]),
    0)
print("Score: ", sum(m)/df['instance_id'].nunique())

nl = df['level'].nunique() # number of levels
m = m.reshape((len(df)//nl, nl))
m

Score:  0.2222222222222223


array([[ 0.        ,  0.        , -0.5       ],
       [ 0.        ,  0.        ,  0.16666667],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.33333333,  0.        ],
       [-0.16666667,  0.33333333,  0.16666667],
       [ 0.5       ,  0.33333333,  0.16666667]])

In [97]:
# macro accuracy per level
ma = []
for level, group in df.groupby('level'):
    print("level:",level)
    print(metrics.accuracy_score(group['label'],group['prediction']))

level: 0
0.16666666666666666
level: 1
0.5
level: 2
0.8333333333333334


In [129]:

def accuracy_score(y_true, y_pred, balanced=True, adjusted=False):
    """Implementation based on https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html
    """
    C = metrics.confusion_matrix(y_true, y_pred)
    if balanced:
        with np.errstate(divide="ignore", invalid="ignore"):
            per_class = np.diag(C) / C.sum(axis=1)
        if np.any(np.isnan(per_class)):
            # warnings.warn("y_pred contains classes not in y_true")
            per_class = per_class[~np.isnan(per_class)]
        score = np.mean(per_class)
        if adjusted:
            n_classes = len(per_class)
            if n_classes > 1: # bug fix if only one y_true class has been detected
                chance = 1 / n_classes
                score -= chance
                score /= 1 - chance
    else:
        score = np.diag(C).sum() / C.sum()
    return float(score)

ma = []
for level, group in df.groupby('level'):
    print("level:",level)
    labels = utils.multiclass.unique_labels(group['label'],group['prediction'])
    print(metrics.accuracy_score(group['label'],group['prediction']))
    print(accuracy_score(group['label'],group['prediction']))

level: 0
0.16666666666666666
0.5
level: 1
0.5
0.7
level: 2
0.8333333333333334
0.8333333333333334
